In [ ]:
import os
import sys
import shutil
import argparse
import IPython 
from PIL import Image, ImageFont, ImageDraw
from enum import Enum
from typing import Callable, List, Optional, Tuple, Union
from functools import partial

import torch
import torch.nn as nn
import torchvision
from torchvision.datasets import VisionDataset
from torchvision.transforms import transforms
import numpy as np
import pandas as pd
import skimage
from scipy import sparse
import matplotlib.pyplot as plt 
import torchxrayvision as xrv

from dinov2.models.unet import UNet
from dinov2.data import SamplerType, make_data_loader, make_dataset
from dinov2.data.datasets import NIHChestXray, MC
from dinov2.data.loaders import make_data_loader
from dinov2.data.transforms import make_segmentation_target_transform, make_segmentation_transform
from dinov2.eval.setup import setup_and_build_model
from dinov2.eval.utils import ModelWithIntermediateLayers, ModelWithNormalize, evaluate, extract_features
from dinov2.eval.metrics import build_segmentation_metrics, MetricAveraging
from dinov2.utils import show_image_from_tensor
from dinov2.eval.segmentation import setup_decoders, DINOV2Encoder, LinearDecoder
from fvcore.common.checkpoint import Checkpointer, PeriodicCheckpointer

In [ ]:
args = argparse.Namespace(config_file='dinov2/configs/eval/vits14_pretrain.yaml', pretrained_weights='models/dinov2_vits14_pretrain.pth', output_dir='results/NIH/dinov2_vits14/knn', opts=[], train_dataset_str='MC:split=TRAIN:root=/mnt/z/data/MC/train', val_dataset_str='MC:split=VAL:root=/mnt/z/data/MC/test', nb_knn=[5, 20, 50, 100, 200], temperature=0.07, gather_on_cpu=False, batch_size=8, n_per_class_list=[-1], n_tries=1, ngpus=1, nodes=1, timeout=2800, partition='learnlab', use_volta32=False, comment='', exclude='')
model, autocast_dtype = setup_and_build_model(args)

In [ ]:
transform = make_segmentation_transform(resize_size=448)
target_transform = make_segmentation_target_transform(resize_size=448)

train_dataset = make_dataset(
    dataset_str=args.train_dataset_str,
    transform=transform,
    target_transform=target_transform
)
val_dataset = make_dataset(
    dataset_str=args.val_dataset_str,
    transform=transform,
    target_transform=target_transform,
)

sampler_type = SamplerType.INFINITE

In [ ]:
embed_dim = model.embed_dim
decoders, optim_param_groups = setup_decoders(
    embed_dim,
    [1e-6, 2e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4, 2e-4, 5e-4, 1e-3, 2e-3, 5e-3, 1e-2, 5e-2, 1e-1],
    3,
)

In [ ]:
output_dir = "models/trained_heads/segmentation_linear/model_final.pth"
checkpointer = Checkpointer(decoders, output_dir)
start_iter = checkpointer.resume_or_load(output_dir, resume=True).get("iteration", -1) + 1

In [ ]:
autocast_ctx = partial(torch.cuda.amp.autocast, enabled=True, dtype=autocast_dtype)
decoder = decoders.module.decoders_dict["segmentor_lr_0_1000000000"]
feature_model = DINOV2Encoder(model, autocast_ctx=autocast_ctx)

In [ ]:
highlight_multipler = 50
metric = build_segmentation_metrics(average_type=MetricAveraging.SEGMENTATION_METRICS, num_labels=3).cuda()
result_message = ""
for image, target in val_dataset:
    image, target = image.cuda(non_blocking=True).unsqueeze(0), target.cuda(non_blocking=True).unsqueeze(0)
    with torch.no_grad(): 
        features=model.forward_features(image)['x_norm_patchtokens']
    logits = decoder(features)
    logits = torch.nn.functional.interpolate(logits, size=448, mode="bilinear", align_corners=False)
    prediction = logits.argmax(dim=1)

    results = metric(prediction, target)

    target = target.squeeze()
    prediction = prediction.squeeze()

    concated = torch.cat((target, prediction), dim=-1)

    prediction = (prediction * highlight_multipler).cpu()
    target = (target * highlight_multipler).cpu()
    concated = (concated.unsqueeze(0).type(torch.int32) * highlight_multipler).cpu()
    H, W = concated.squeeze().shape

    pil_image = torchvision.transforms.ToPILImage()(concated)
    pil_image = pil_image.convert("L") # Convert to Grayscale

    draw = ImageDraw.Draw(pil_image)
    _, _, w, h = draw.textbbox((0, 0), "Target")
    draw.text(((W-w)*0.25, H*0.025), "Target", fill=255)

    _, _, w, h = draw.textbbox((0, 0), "Prediction")
    draw.text(( (W-w) * 0.75, H*0.025), "Prediction", fill=255)

    for m, r in dict(results).items():
        result_message += f"{m}: {float(r):.3f} "

    draw.text((0, 0), result_message, fill=255)

    pil_image.save("hello.png")
    print(metric.compute())
    break